## Analysis of waist-to-hip ratio (WHR)

This notebook applies the `Get_Job_Script.ipynb` to automatically generate the sbatch scripts to run in Yale's cluster. The end result is to apply [various LMM workflows](https://github.com/statgenetics/UKBB_GWAS_dev/tree/master/workflow) to perform association analysis in the INT-WHR trait, do clumping analysis and extract associated regions.

## File paths on Yale cluster

- Genotype files in PLINK format:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv`
- Genotype files in BGEN format:
`SAY/dbgapstg/scratch/UKBiobank/genotype_files/ukb39554_imputeddataset/`
- Summary stats for imputed variants BOLT-LMM:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data`
- Summary stats for inputed variants FastGWA:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data`
- Phenotype files:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis`
- Relationship file:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620`

## 07/01/20 analysis

On the cluster, open up this notebook using the JupyterLab server you set up via the ssh channel, then run the following cells,

### Bash variables for workflow configurations

In [50]:
# Common variables
tpl_file=../farnam.yml
bfile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated082020removedwithdrawnindiv.bed
sampleFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb32285_imputedindiv.sample
bgenFile=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb_imp_chr{1..22}_v3.bgen`
unrelated_samples=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620
formatFile_fastgwa=~/project/UKBB_GWAS_dev/data/fastGWA_template.yml
formatFile_bolt=~/project/UKBB_GWAS_dev/data/boltlmm_template.yml
formatFile_saige=~/project/UKBB_GWAS_dev/data/saige_template.yml
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
container_marp=/gpfs/gibbs/pi/dewan/data/UKBiobank/marp.sif
# LMM directories
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/INT-WHR
lmm_dir_bolt=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/INT-WHR
lmm_dir_saige=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/SAIGE_results/results_imputed_data/INT-WHR
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_INT-WHR-fastgwa.sbatch
lmm_sbatch_bolt=../output/$(date +"%Y-%m-%d")_INT-WHR-bolt.sbatch
lmm_sbatch_saige=../output/$(date +"%Y-%m-%d")_INT-WHR-saige.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/normalized_phenotypes/phenotypes_UKB/UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-WHR_withagesex_042020
## LMM variables 
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/normalized_phenotypes/UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-WHR_withagesex_042020
LDscoresFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/LDSCORE.1000G_EUR.tab.gz
geneticMapFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genetic_map_hg19_withX.txt.gz
phenoCol=INT-WHR
covarCol=SEX
covarMaxLevels=10
qCovarCol="AGE BMI"
numThreads=20
bgenMinMAF=0.001
bgenMinINFO=0.8
lmm_job_size=1
### Specific to SAIGE
bgenMinMAC=4
trait_type=quantitative
loco=TRUE
sampleCol=IID
# LD clumping directories
clumping_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/INT-WHR
clumping_sos=../workflow/LD_Clumping.ipynb
clumping_sbatch=../output/$(date +"%Y-%m-%d")_INT-WHR_ldclumping.sbatch
## LD clumping variables
bfile_ref=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.1210.ref_geno
# For sumtastsFiles if more than one provide each path
# In this case analysis INT-WHR and asthma
sumstatsFiles="/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/INT-WHR/ukb_imp_v3.UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-WHR_withagesex_042020.BoltLMM.snp_stats.all_chr.gz \
               /gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/asthma/ukb_imp_v3.Asthma_casesbyICD10codesANDselfreport_controlsbyselfreportandicd10_noautoimmuneincontrols.fastGWA.snp_stats.all_chr.gz"
ld_sample_size=1210
clump_field=P
clump_p1=5e-08
clump_p2=1
clump_r2=0.2
clump_kb=2000
clump_annotate=BP
numThreads=20
clump_job_size=1
# Region extraction directories
extract_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/region_extraction/INT-WHR
extract_sos=../workflow/Region_Extraction.ipynb
extract_sbatch=../output/$(date +"%Y-%m-%d")_INT-WHR-region.sbatch
## Region extraction variables
region_file=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/INT-WHR/*.clumped_region
geno_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/UKBB_bgenfilepath.txt
sumstats_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/INT-WHR/ukb_imp_v3.UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-WHR_withagesex_042020.BoltLMM.snp_stats.all_chr.gz
extract_job_size=10

### BoltLMM job

In [17]:
#Running the workflow for INT-WHR trait. Here I used age, sex and BMI as covariates.
lmm_args="""boltlmm
    --cwd $lmm_dir_bolt
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_bolt 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ../workflow/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_bolt \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-07-17_INT-WHR-bolt.sbatch
INFO: Workflow farnam (ID=d38bbddbb1f90efa) is executed successfully with 1 completed step.


## FastGWA job

In [ ]:
lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ../workflow/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"

## SAIGE job

In [ ]:
lmm_args="""SAIGE
    --cwd $lmm_dir_saige 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_saige 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --bgenMinMAC $bgenMinMAC
    --trait_type $trait_type
    --loco $loco
    --sampleCol $sampleCol
    --job_size $lmm_job_size
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ../workflow/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_saige \
    --args "$lmm_args"

### LD clumping job

In [51]:
clumping_args="""default 
    --cwd $clumping_dir 
    --bfile $bfile
    --bfile_ref $bfile_ref
    --bgenFile $bgenFile
    --sampleFile $sampleFile 
    --sumstatsFiles $sumstatsFiles 
    --unrelated_samples $unrelated_samples 
    --ld_sample_size $ld_sample_size 
    --clump_field $clump_field
    --clump_p1 $clump_p1 
    --clump_p2 $clump_p2 
    --clump_r2 $clump_r2 
    --clump_kb $clump_kb 
    --clump_annotate $clump_annotate 
    --numThreads $numThreads 
    --job_size $clump_job_size
"""

sos run ../workflow/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $clumping_sos \
    --to-script $clumping_sbatch \
    --args "$clumping_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-07-17_INT-WHR_ldclumping.sbatch
INFO: Workflow farnam (ID=8a9b20dae6b34876) is executed successfully with 1 completed step.


In [47]:
%preview ../output/2020-07-17_INT-WHR_ldclumping.sbatch -l -1

> ../output/2020-07-17_INT-WHR_ldclumping.sbatch (3.9 KiB):

32 lines (-1 displayed, see --limit)

#!/bin/bash
#SBATCH --partition general
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 4
#SBATCH --mem 16G
#SBATCH --time 5-0:00:00
#SBATCH --job-name ../output/2020-07-17_INT-WHR_ldclumping
#SBATCH --output ../output/2020-07-17_INT-WHR_ldclumping-%J.out
#SBATCH --error ../output/2020-07-17_INT-WHR_ldclumping-%J.log

sos run /gpfs/ysm/project/dewan/dc2325/UKBB_GWAS_DEV/workflow/LD_Clumping.ipynb \
    default \
    --cwd /SAY/dbgapstg/scratch/UKBiobank/results/LD_clumping/INT-WHR \
    --bfile /SAY/dbgapstg/scratch/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.bed\
    --bfile_ref /SAY/dbgapstg/scratch/UKBiobank/results/LD_clumping/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.1210.ref_geno\
    --bgenFile /SAY/dbgapstg/scratch/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb_imp_chr1_v3.bgen /SAY/dbgapstg/scratch/UKBiobank/genotype_files/ukb39554_imputeddat

>>> ../output/2020-07-17_INT-WHR_ldclumping.sbatch:

bash: ../output/2020-07-17_INT-WHR_ldclumping.sbatch: Permission denied


### Region extract job

In [19]:
extract_args="""default
    --cwd $extract_dir
    --region-file $region_file
    --pheno-path $phenoFile
    --geno-path $geno_path
    --bgen-sample-path $sampleFile
    --sumstats-path $sumstats_path
    --format-config-path $formatFile
    --unrelated-samples $unrelated_samples
"""

sos run ../workflow/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $extract_sos \
    --to-script $extract_sbatch \
    --args "$extract_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-07-17_INT-WHR-region.sbatch
INFO: Workflow farnam (ID=e9d440337019346e) is executed successfully with 1 completed step.
